In [ ]:
Delta = 0.05

# FORESEE - Inelastic electric dipole DM Delta=0.0500

### Load Libraries 

In [ ]:
import numpy as np
import sys
import os

src_path = "../../src"
sys.path.append(src_path)
from foresee import Foresee, Utility, Model
from main import el_dip_sigma_chi0Nucleus_chi1Nucleus_analyt, el_dip_sigma_chi0e_chi1e_analyt, el_dip_Gamma_chi1_chi0g, el_dip_Gamma_chi1_chi0ll
from constants import *


from matplotlib import pyplot as plt
import matplotlib.tri as tri

plt.rc('text', usetex=True)
plt.rcParams['figure.dpi'] = 400

plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amssymb}"]
font = {'family': 'serif', 'serif': ['computer modern roman']}
plt.rcParams['text.latex.preamble'] = [r"\usepackage{siunitx}"]

plt.rc('font', **font)

SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 1. Initialization 

All function that we will need are included in the FORESEE package. We start by simply initializing it: 

In [ ]:
foresee = Foresee()

In [ ]:
# beam energy for NuCal
energy = "0.069"
modelname = "Inelastic_electric_dipole_DM_Delta_0.0500"
model = Model(modelname)

nsample = 100

num_of_masses = 21
masses = np.logspace(-2, np.log10(4e-1), num_of_masses)
masses_chi0 = masses / (1 + Delta)

num_of_couplings = 31

### Generate LLP spectum

#### Mesons decays

**MDM**

PDG codes: $\gamma$ = 22, $\pi_0$ = 111, $\eta$ = 221, $\eta^\prime$ = 331.

Pseudoscalar-meson decays:

\begin{equation} 
	\begin{aligned}
		\text{MDM:}\quad     
			&  \frac{d{\rm BR}_{P \rightarrow \gamma \bar{\chi}_0 \chi_1}}{dq^2 d\cos\theta} = {\rm BR}_{P\rightarrow \gamma \gamma}  \!\times\!\!  \frac{\left(q^2-M^2\right)^3 \left(q^2-(m_{\chi_0}-m_{\chi_1})^2\right) \sqrt{-2 m_{\chi_0}^2 \left(m_{\chi_1}^2+q^2\right)+m_{\chi_0}^4+\left(m_{\chi_1}^2-q^2\right)^2}}{8 \pi^2 \Lambda_m^2 M^6 q^6} \Big[-\cos (2 \theta) \left((m_{\chi_0}+m_{\chi_1})^2-q^2\right)-3 (m_{\chi_0}+m_{\chi_1})^2-q^2 \Big]
	\end{aligned}
\end{equation} 



Vector-meson decays:

\begin{equation} 
	\begin{aligned}
		\!\!\text{MDM:}\     
			&\frac{{\rm BR}_{V \rightarrow \bar{\chi}_0 \chi_1}}{{\rm BR}_{V \rightarrow ee}} \!=\! \frac{(M +m_{\chi_0}-m_{\chi_1}) (M - m_{\chi_0}+m_{\chi_1}) \left(M ^2+2 (m_{\chi_0}+m_{\chi_1})^2\right) \sqrt{\left(-M ^2+m_{\chi_0}^2+m_{\chi_1}^2\right)^2-4 m_{\chi_0}^2 m_{\chi_1}^2}}{2\pi\alpha \Lambda_m^2 M  \left(M ^2+2 m_e^2\right) \sqrt{M ^2-4 m_e^2}}, \!\! \\
	\end{aligned}
\end{equation} 


#### Generate LLP spectrum

In [ ]:
# NuCal

# V(p0) -> chi0(p1) + chi1(p2)
# p1**2 = m1**2 = mchi0**2
# p2**2 = m2**2 = mchi1**2

model.add_production_2bodydecay(
    pid0 = "113", # rho
    pid1 = "0",   # pid1=0 means mass_pid1 (m1) is passed by mass_llp0; mass_pid2=m2=mass
    br="1.7e18*4.72e-5 * coupling**2 * (1*(m0**2 + 2*(m1 - m2)**2)*(m0 - m1 - m2)*(m0 + m1 + m2)* np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (2.*ALPHAEM*m0*np.sqrt(m0**2 - 4*M_ELECTRON**2)*(m0**2 + 2*M_ELECTRON**2)*np.pi)",
    generator="Pythia",
    energy = energy,
    nsample = nsample,
)

model.add_production_2bodydecay(
   pid0 = "223", # omega
   pid1 = "0",
   br="1.7e18*7.38e-5 * coupling**2 * (1*(m0**2 + 2*(m1 - m2)**2)*(m0 - m1 - m2)*(m0 + m1 + m2)* np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (2.*ALPHAEM*m0*np.sqrt(m0**2 - 4*M_ELECTRON**2)*(m0**2 + 2*M_ELECTRON**2)*np.pi)",
   generator="Pythia",
   energy = energy,
   nsample = nsample,
)

model.add_production_2bodydecay(
   pid0 = "333", # phi
   pid1 = "0",
   br="1.7e18*2.98e-4 * coupling**2 * (1*(m0**2 + 2*(m1 - m2)**2)*(m0 - m1 - m2)*(m0 + m1 + m2)* np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (2.*ALPHAEM*m0*np.sqrt(m0**2 - 4*M_ELECTRON**2)*(m0**2 + 2*M_ELECTRON**2)*np.pi)",
   generator="Pythia",
   energy = energy,
   nsample = nsample,
)

model.add_production_2bodydecay(
    pid0 = "443", # J/ψ
    pid1 = "0",
    br="1.7e18*0.0597 * coupling**2 * (1*(m0**2 + 2*(m1 - m2)**2)*(m0 - m1 - m2)*(m0 + m1 + m2)* np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (2.*ALPHAEM*m0*np.sqrt(m0**2 - 4*M_ELECTRON**2)*(m0**2 + 2*M_ELECTRON**2)*np.pi)",
    generator="Pythia",
    energy = energy,
    nsample = nsample,
)

In [ ]:
model.set_ctau_1d(filename="model/el_dip_ctau_chi1_Delta_0.0500.txt", coupling_ref=1)

branchings = [
    [ "el_dip_BR_chi1_chi0g_Delta_0.0500", "black", "solid", r"$\chi_0\gamma$", 0.110, 0.30 ],
    [ "el_dip_BR_chi1_chi0ee_Delta_0.0500", "red", "solid", r"$\chi_0 e^+ e^-$", 0.110, 0.016 ],
]

model.set_br_1d(
    modes=[channel for channel, _, _, _, _, _ in branchings],
    filenames=[ "model/br/" + channel + ".txt" for channel, _, _, _, _, _ in branchings ],
)

foresee.set_model(model=model)

In [ ]:
# mass_llp1 = 0.01
# mass_llp0 = mass_llp1 / (1+Delta)

# # plot the test spectrum
# plt_1, plt_2 = foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1e-4, do_plot=True, save_file=False)
# plt_1.savefig("./output/test_LLP_spect_plt_1.pdf")
# plt_2.savefig("./output/test_LLP_spect_plt_2.pdf")
# plt_1.show()
# plt_2.show()

In [ ]:
from timeit import default_timer as timer

for count, mass in enumerate(masses):
    start = timer()
    mass_llp1 = mass
    mass_llp0 = mass_llp1 / (1+Delta)

    foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1, detector="NuCal", stat_cuts_llp0="p_llp0.e>1.0", stat_cuts_llp1="p_llp1.e>1.0")

    end = timer()
    time_length_sec = end - start

    count += 1
    time_length_sec_total = time_length_sec * num_of_masses
    print("%.2f%% done, " % float(count / num_of_masses * 100), "approx. total run time : %.1f m, " % float(time_length_sec_total / 60), "approx. waiting time: %.1f m" % float(time_length_sec_total * (1 - count / num_of_masses) / 60))

In [ ]:
# productions = [
#     ["111"    , None      , "firebrick"   , r"$\pi$"         ],   
#     ["221"    , None      , "red"         , r"$\eta$"        ],   
#     ["331"    , None , "salmon"      , r"$\eta'$"       ],  
#     ["113"    , None , "dodgerblue"  , r"$\rho$"        ],   
#     ["223"    , None , "blue"        , r"$\omega$"      ],   
#     ["333"    , None , "deepskyblue" , r"$\phi$"        ],  
#     ["443"    , None  , "gold"        , r"$J/\psi$"      ],   
#     # ["100443" , None  , "orange"      , r"$\psi(2S)$"    ],  
#     # ["553"    , None  , "green"       , r"$\Upsilon(1S)$"],   
#     # ["100553" , None  , "limegreen"   , r"$\Upsilon(2S)$"],  
#     # ["200553" , None  , "lime"        , r"$\Upsilon(3S)$"],  
# ]


# plot = foresee.plot_production(
#     masses = masses, 
#     productions = productions,
#     condition="True", 
#     xlims=[0.01,2],ylims=[10**0,10**10], 
#     xlabel=r"Mass [GeV]", 
#     ylabel=r"Production Rate $\sigma/\epsilon^2$ [pb]",
#     legendloc=(1.02,1.02),
#     fs_label=12,
# )

# plot.savefig("output/Production_channels_NuCal.pdf")
# plot.show()

## 2. NuCal

In [ ]:
#specify setup
luminosity, distance = 1/1000, 64
setup, selection, channels, length = "NuCal_chi0g", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "el_dip_BR_chi1_chi0g_Delta_0.0500" ], 23
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)

list_nevents = []
for mass in masses:
    mass_chi0 = mass / (1 + Delta)
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings), detector="NuCal", preselectioncuts="p>3", check_visible_energy_two_body_decay=True, mass_1=mass_chi0, mass_2=0.0, E_th=3,)

    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses_chi0, couplings, list_nevents])